In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from IPython.display import display, HTML

# Extração dos dados de WEGE3.SA do Yahoo Finance
wege3 = yf.download('WEGE3.SA', period='max')

# Convertendo o índice para datetime e removendo os horários
wege3.index = pd.to_datetime(wege3.index).tz_localize(None).date
wege3.index.name = 'Date'

# Criando uma cópia dos dados originais para comparação
wege3_original = wege3.copy()

# Filtrando os dados a partir de maio de 2024
wege3_original = wege3_original[wege3_original.index >= datetime.date(2024, 5, 1)]
wege3_filtered = wege3[wege3.index >= datetime.date(2024, 5, 1)].copy()

# Calculando o resultado percentual
wege3_filtered.loc[:, 'Resultado%'] = (wege3_filtered['Close'] - wege3_filtered['Open']) / wege3_filtered['Open'] * 100

# Inserindo a coluna 'Ativo'
wege3_filtered.insert(0, 'Ativo', 'WEGE3')

# Arredondando os dados OHLC
wege3_filtered.iloc[:, 1:] = wege3_filtered.iloc[:, 1:].apply(lambda x: round(x, 2))

# Configurando a formatação global dos floats
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Detectando colunas do tipo float
lista_colunas_float = []
for i in range(0, len(wege3_filtered.columns)):
    if type(wege3_filtered.iloc[1, i]) == np.float64:
        lista_colunas_float.append(i)

# Aplicando formatação e estilo
styled_df = wege3_filtered.style.format('{:.2f}', subset=wege3_filtered.columns[lista_colunas_float])

# Função para aplicar formatação condicional
def color_negative_red(value):
    color = '#800000' if value < 0 else '#008200'
    return f'font-weight: bold; color: white; background-color: {color}'

# Aplicando formatação condicional para a coluna 'Resultado%'
styled_df = styled_df.map(color_negative_red, subset=["Resultado%"])

# Configurando estilos adicionais para a tabela
styled_df = styled_df.set_table_styles(
    [{"selector": "td", "props": "text-align: center"},
     {"selector": "th", "props": "text-align: center; font-weight: bold; background-color: blue; color: lightgray; font-size: 12pt"},
     {"selector": "th.index_name", "props": "text-align: center; font-weight: bold; background-color: blue; color: lightgray; font-size: 12pt"},
     {"selector": "th.row_heading", "props": "text-align: center; font-weight: bold; background-color: lightblue; color: black; font-size: 10pt"}])

# Exibindo os DataFrames lado a lado
html = f"""
<div style="display: flex; justify-content: space-around;">
    <div>
        <h3 style="color: yellow;">Dados Originais</h3>
        {wege3_original.tail(len(wege3_filtered)).to_html()}
    </div>
    <div>
        <h3 style="color: yellow;">Dados Formatados</h3>
        {styled_df.to_html()}
    </div>
</div>
"""

display(HTML(html))


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-05-02,39.69,40.00,38.00,38.85,38.85,12268300
2024-05-03,38.99,38.99,38.24,38.77,38.77,5092600
2024-05-06,38.61,38.92,38.40,38.63,38.63,2810000
2024-05-07,38.60,39.21,38.50,38.90,38.90,4895900
2024-05-08,38.70,39.09,38.56,39.09,39.09,3759000
2024-05-09,38.67,38.93,38.27,38.90,38.90,4949800
2024-05-10,38.86,39.11,38.72,38.97,38.97,3462100
2024-05-13,39.09,39.32,38.50,39.08,39.08,11032700
2024-05-14,39.01,39.79,38.99,39.61,39.61,9947400
